In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import scipy.sparse as sp
import torch

from rlaopt.sparse import SparseCSRTensor

In [3]:
torch_precision = torch.float64
np_precision = np.float32 if torch_precision == torch.float32 else np.float64

torch.set_default_dtype(torch_precision)

In [4]:
X = sp.load_npz("yelp_train.npz").astype(np_precision)
# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

X_torch = SparseCSRTensor(data=X, device=device)

/home/pratikr/rlaopt/rlaopt/sparse/sparse_tensor.py:122: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  data = torch.sparse_csr_tensor(


In [5]:
X_torch.data

tensor(crow_indices=tensor([        0,       343,       408,  ...,
                            768737453, 768737550, 768737914]),
       col_indices=tensor([   1451,    2230,   12179,  ..., 2071645, 2071654,
                           2073613]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]), size=(5038676, 2084724),
       nnz=768737914, layout=torch.sparse_csr)

In [6]:
X_torch.device

device(type='cpu')

In [7]:
X_torch.shape

torch.Size([5038676, 2084724])

In [8]:
X_torch.dtype

torch.float64

In [9]:
idx = torch.tensor(np.arange(1024), dtype=torch.int64, device=device)
X_torch_sampled = X_torch[idx]
X_torch_sampled.data

tensor(crow_indices=tensor([     0,    343,    408,  ..., 153449, 153570,
                            154031]),
       col_indices=tensor([   1451,    2230,   12179,  ..., 2075547, 2077673,
                           2078233]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]), size=(1024, 2084724),
       nnz=154031, layout=torch.sparse_csr)

In [10]:
X_torch_sampled.shape

torch.Size([1024, 2084724])

In [11]:
b = torch.randn(X_torch.shape[1], 10, device=device)
print(X_torch @ b)
print(X_torch_sampled @ b)

tensor([[-14.7711,   7.0236,  -8.6730,  ...,  28.5785,  17.2703, -18.9184],
        [ 11.0300,   9.1180,  11.6621,  ..., -12.2571,  13.5049,   7.5169],
        [ -5.3530,  -1.4366,   4.8977,  ...,   7.0360,  17.0810, -25.2769],
        ...,
        [ 16.0472,   3.8170,  28.0368,  ...,   6.7631, -17.5206,  17.6966],
        [  5.1741, -19.3341,   1.1223,  ...,   7.0862,  -3.3971, -23.2195],
        [  7.7299,  20.4977,  12.9293,  ..., -11.9673,  26.9342, -20.9296]])
tensor([[-14.7711,   7.0236,  -8.6730,  ...,  28.5785,  17.2703, -18.9184],
        [ 11.0300,   9.1180,  11.6621,  ..., -12.2571,  13.5049,   7.5169],
        [ -5.3530,  -1.4366,   4.8977,  ...,   7.0360,  17.0810, -25.2769],
        ...,
        [ -8.8887,  19.6059,  13.9887,  ...,  -8.5609, -10.1209,   4.7427],
        [  9.8967,   1.1939,  -0.3986,  ...,  -5.4618,   6.6153,  29.6980],
        [  1.0828,  -3.3235, -23.4061,  ...,   5.5324,   4.3511,   0.7197]])


In [12]:
X_torch_T = X_torch.T
print(X_torch_T.data)
print(X_torch_T.shape)
print(X_torch_T.dtype)

tensor(ccol_indices=tensor([        0,       343,       408,  ...,
                            768737453, 768737550, 768737914]),
       row_indices=tensor([   1451,    2230,   12179,  ..., 2071645, 2071654,
                           2073613]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]), size=(2084724, 5038676),
       nnz=768737914, layout=torch.sparse_csc)
torch.Size([2084724, 5038676])
torch.float64


In [13]:
d = torch.randn(X_torch_T.shape[1], device=device)
result = X_torch_T @ d

In [14]:
result

tensor([89.4837, -4.7829, 11.0437,  ...,  1.6481,  3.4253, -3.4608])

In [15]:
d_np = d.cpu().numpy()
result_np = X.T @ d_np

In [16]:
result_np

array([89.48374268, -4.78292312, 11.0436945 , ...,  1.64808316,
        3.42530148, -3.46084915], shape=(2084724,))

In [17]:
np.allclose(result.cpu().numpy(), result_np)

True